In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from Data import*

bom_df = pd.read_excel("Bill of Material (mrp.bom)updated.xlsx")

DataMgr = DataManager()
DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct (dbmgr,product):

    if (product.find("[")>-1) and (product.find("]")>-1):
        productnumber = product[product.find("[")+1:product.find("]")];
        productname = product[product.find("]")+1:];
        if not "["+productnumber+"]_"+productname in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),"["+productnumber+"]_"+productname,productnumber,0);
            return newproduct
        else:
            return dbmgr.getProducts()["["+productnumber+"]_"+productname]
    else:
        if not product in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),product,product,0);
            return newproduct
        else:
            return dbmgr.getProducts()[product]

def MakeOperation(dbmgr,operation,prod,resource):
    myresource=None;
    if not resource in dbmgr.getResources():
        if "manual" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Manual",resource,15);
            dbmgr.getResources()[resource]=myresource;
        elif "Outsourced" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Outsourced",resource,10000);
            dbmgr.getResources()[resource]=myresource;
            print(resource)
        else:
            myresource = Resource(len(dbmgr.getResources()),"Machine",resource,15);
            dbmgr.getResources()[resource]=myresource;
    else:
        myresource = dbmgr.getResources()[resource]; 

    optimestr= r['BoM Lines/Operations/Duration'];

    optime = 0
    
    if isinstance(optimestr, (int, float)):
        optime = float(1/60)*float(optimestr) # time assumed to be minute..

    newopr = Operation(len(dbmgr.getOperations()),operation+"_"+prod.getName(),optime)

    if myresource !=None:
        myresource.getOperations().append(newopr)
        newopr.getRequiredResources().append(myresource)

    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()]=newopr;
        
    return newopr

def SaveInstance(DataMgr,file):

        # Save Products.
        products_df = pd.DataFrame(columns= ["ProductID", "ProductNumber","Name","StockLevel"])
        precedences_df = pd.DataFrame(columns= ["PredecessorID","SuccessorID","Multiplier"])
        prodrops_df = pd.DataFrame(columns= ["ProductID","OperationID","OperationIndex"])
        operations_df = pd.DataFrame(columns= ["OperationID","Name","ProcessTime"])
        opsres_df = pd.DataFrame(columns= ["OperationID","ResourceID"])
        resources_df = pd.DataFrame(columns= ["ResourceID","ResourceType","Name","DailyCapacity"])
        orders_df = pd.DataFrame(columns= ["OrderID","ProductID","ProductName","Name","Quantity","Deadline"])
 
        
        for name,myprod in DataMgr.Products.items():
            products_df.loc[len(products_df)] = {"ProductID":myprod.getID(), "ProductNumber":myprod.getPN(),"Name":myprod.getName(),"StockLevel":myprod.getStockLevel()}

            
            for pred in myprod.getPredecessors():
                precedences_df.loc[len(precedences_df)] = {"PredecessorID":pred.getID(),"SuccessorID":myprod.getID(),"Multiplier":1}
 
            oprind = 0
            for opr in myprod.getOperations(): 
                prodrops_df.loc[len(prodrops_df)] = {"ProductID":myprod.getID(),"OperationID":opr.getID(),"OperationIndex":oprind}
                oprind+=1 

        for opname,opr in DataMgr.Operations.items():
            operations_df.loc[len(operations_df)]= {"OperationID":opr.getID(),"Name":opr.getName(),"ProcessTime":opr.getProcessTime()}
            
            for res in opr.getRequiredResources(): 
                opsres_df.loc[len(opsres_df)]= {"OperationID":opr.getID(),"ResourceID":res.getID()}
        

        for resname,res in DataMgr.Resources.items():
            resources_df.loc[len(resources_df)] ={"ResourceID":res.getID(),"ResourceType":res.getType(),"Name":res.getName(),"DailyCapacity":res.getDailyCapacity()}
 
        
        for ordname,ordr in DataMgr.CustomerOrders.items():
            orders_df.loc[len(orders_df)]={"OrderID":ordr.getID(),"ProductID":ordr.getProduct().getID(),"ProductName":ordr.getProduct().getName(),"Name":ordr.getName(),"Quantity":ordr.getQuantity(),"Deadline":ordr.getDeadLine()}

    


        folder = 'UseCases'; casename = file
        path = folder+"\\"+casename
        isExist = os.path.exists(path)

        if not isExist:
            os.makedirs(path)
       
        
        filename = 'Products.csv'; path = folder+"\\"+casename+"\\"+filename;  fullpath = os.path.join(Path.cwd(), path) 
        products_df.to_csv(path, index=False)
        filename = 'Precedences.csv';path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        precedences_df.to_csv(fullpath, index=False)
        filename = 'ProductsOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        prodrops_df.to_csv(fullpath, index=False)
        filename = 'ResourcesOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        opsres_df.to_csv(fullpath, index=False)
        filename = 'Operations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        operations_df.to_csv(fullpath, index=False)
        filename = 'Resources.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        resources_df.to_csv(fullpath, index=False)
        filename = 'CustomerOrders.csv'; path = folder+"\\"+casename+"\\"+filename;fullpath = os.path.join(Path.cwd(), path)
        orders_df.to_csv(fullpath, index=False)

        return
    
nopass = 0;
for i,r in bom_df.iterrows():

    productrow = False;
    if nopass > 0:
        nopass -=1
        continue

    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow = True;

    if productrow == True:
        newproduct = MakeProduct(DataMgr,r['Product'])
        DataMgr.getProducts()[newproduct.getName()] = newproduct;

        if not pd.isnull(r['BoM Lines/Component']):
            
            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'])

            quantity = r['BoM Lines/Quantity']

            newproduct.getMPredecessors()[rawprod] = quantity
            newproduct.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newproduct)

            DataMgr.getProducts()[rawprod.getName()] = rawprod

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                if bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')] == "OUT - Outsourced activity":
                    myopr = MakeOperation(DataMgr,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')]+"_"+bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')])
                    newproduct.getOperations().append(myopr)
                    print(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')]+"_"+bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')])
                else:
                    myopr = MakeOperation(DataMgr,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')])
                    newproduct.getOperations().append(myopr)
                
               
   
            j+=1 
        nopass=len(newproduct.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"_("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))
print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))

SaveInstance(DataMgr,'Test_Bryan')

    

    
    
        

    

In [ ]:
import os
online_version = False;
from DBMain import*

#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); tab_2 = VisMgr.generateCOTAB(); tab_3 = VisMgr.generatePLTAB()
tab_5 = VisMgr.generatePSschTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_2,tab_3,tab_5])
tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Production System'); tab_set.set_title(2, 'Customer Orders');
tab_set.set_title(3, 'Production Planning'); tab_set.set_title(4, 'Production Schedule');tab_set

In [ ]:
DataMgr.getResources()